# **Beam Search**

✔ Beam search, greedy search'den farklı olarak yalnızca en olası bir sonraki kelimeyi değil, "n" en olası kelimeyi değerlendirir.

✔ n beam sayısını temsil eder.

✔ Dizi skoru, her kelimenin log olasılıklarının toplamıyla hesaplanır ve uzunlukla normalize edilir. Bu yöntemle, bir cümleyi tamamlamak için daha geniş bir olasılık aralığı göz önünde bulundurulabilir.


In [2]:
from transformers import GPT2LMHeadModel, AutoTokenizer
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = GPT2LMHeadModel.from_pretrained('gpt2').to(device)
tokenizer = AutoTokenizer.from_pretrained("gpt2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [14]:
import torch
import numpy as np
import networkx as nx

def get_log_prob(logits, token_id):
    """Compute the log probability of a given token."""
    log_probabilities = torch.log_softmax(logits, dim=-1)
    return log_probabilities[token_id.item()].item()

def greedy_sampling(logits, beams):
    """Return the top-k token indices using greedy sampling."""
    return torch.topk(logits, beams).indices

def beam_search(input_ids, length, beams):
    """
    Perform beam search to generate text sequences.

    Args:
        input_ids (Tensor): Tokenized input text.
        length (int): Maximum number of tokens to generate.
        beams (int): Number of sequences to keep at each step.

    Returns:
        List[Tuple[str, float]]: Generated sequences with their scores.
    """
    sequences = [(input_ids, 0)]  # (token sequence, cumulative log probability)

    for _ in range(length):
        all_candidates = []

        for seq, score in sequences:
            outputs = model(seq)
            logits = outputs.logits[0, -1, :]
            top_token_ids = greedy_sampling(logits, beams)

            for token_id in top_token_ids:
                token_score = get_log_prob(logits, token_id)
                new_score = score + token_score
                new_seq = torch.cat([seq, token_id.unsqueeze(0).unsqueeze(0)], dim=-1)
                all_candidates.append((new_seq, new_score))

        # Select the top `beams` sequences
        sequences = sorted(all_candidates, key=lambda x: x[1], reverse=True)[:beams]

    return [(tokenizer.decode(seq.squeeze(), skip_special_tokens=True), score) for seq, score in sequences]

# Example usage
start_text = "I have a"
input_ids = tokenizer.encode(start_text, return_tensors="pt").to(device)

generated_sequences = beam_search(input_ids, length=5, beams=3)

for idx, (sequence, score) in enumerate(generated_sequences):
    print(f"Sequence {idx+1}: {sequence} (Score: {score})")

Sequence 1: I have a lot of respect for the (Score: -7.422885708510876)
Sequence 2: I have a lot of friends who are (Score: -8.3595812022686)
Sequence 3: I have a lot of questions about this (Score: -8.360632807016373)
